In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../secretary')

import pandas as pd
from secretary import todo
import secretary.secretary_bot as sb
from io import StringIO


In [3]:
with open('legalese_email.txt', 'r') as file:
    content = file.read()

message = "From: Jack\n" + content

df_legal = sb.extract_tasks(message)
df_legal

,topic,type,due date,requestor,actor,summary,notes
0,Probate Process - Truist Bank Claim,question,NaN,Marcella Brodhurst,Jack,Confirmation of payment to Truist Bank and for...,"Have you made the payment to Truist? If so, ha..."
1,Probate Process - Safe Deposit Box Inventory,action_item,NaN,Marcella Brodhurst,Jack,Confirm contents removal and closure of accoun...,Please confirm that the contents have been rem...
2,Probate Process - Estate Accounts,question,NaN,Marcella Brodhurst,Jack,Inquiry about closure of accounts and deposit ...,Mark wants to know: have you closed all the ac...
3,Probate Process - Estate Account Income,question,NaN,Marcella Brodhurst,Jack,Request for information on income/interest ear...,Can you tell us how much income and/or interes...
4,Probate Process - Final 1040 Income Tax Return,question,NaN,Marcella Brodhurst,Jack,Inquiry about the final 1040 income tax return,Have you taken care of the final 1040 final in...


In [4]:
todos = todo.Todo()

tasks = [
  {
    "topic": "Shopping",
    "type": "action_item",
    "due date": "2024-04-14",
    "requestor": "Jack",
    "actor": "Jack",
    "summary": "Buy shoes",
    "notes": "I need to buy some cool new kicks."
  },
  {
    "topic": "Grocery Shopping",
    "type": "action_item",
    "due date": "2024-05-14",
    "requestor": "Jack",
    "actor": "Silvia",
    "summary": "Buy eggs",
    "notes": "Silvia needs to buy eggs"
  }
]

df = pd.concat([df_legal, pd.DataFrame(tasks)], ignore_index=True)
todos.add_to_database(df)
todos.df

,topic,type,due date,requestor,actor,summary,notes
0,Probate Process - Truist Bank Claim,question,NaN,Marcella Brodhurst,Jack,Confirmation of payment to Truist Bank and for...,"Have you made the payment to Truist? If so, ha..."
1,Probate Process - Safe Deposit Box Inventory,action_item,NaN,Marcella Brodhurst,Jack,Confirm contents removal and closure of accoun...,Please confirm that the contents have been rem...
2,Probate Process - Estate Accounts,question,NaN,Marcella Brodhurst,Jack,Inquiry about closure of accounts and deposit ...,Mark wants to know: have you closed all the ac...
3,Probate Process - Estate Account Income,question,NaN,Marcella Brodhurst,Jack,Request for information on income/interest ear...,Can you tell us how much income and/or interes...
4,Probate Process - Final 1040 Income Tax Return,question,NaN,Marcella Brodhurst,Jack,Inquiry about the final 1040 income tax return,Have you taken care of the final 1040 final in...
5,Shopping,action_item,2024-04-14,Jack,Jack,Buy shoes,I need to buy some cool new kicks.
6,Grocery Shopping,action_item,2024-05-14,Jack,Silvia,Buy eggs,Silvia needs to buy eggs


In [18]:
comment = "From: Jack\nI still have my shitty old NewBalance sneakers from 2018, but I really do like them."

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.158987
1    0.148805
2    0.152500
3    0.113090
4    0.175671
5    0.378984
6    0.175641
Name: embedding, dtype: float64
Similar task(s):
{
    "5":{
        "completed":false,
        "topic":"Shopping",
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    }
}
Updated task(s):
{
    "5":{
        "completed":false,
        "topic":"Shopping",
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks. *I still have my shitty old NewBalance sneakers from 2018, but I really do like them.*"
    }
}


In [17]:
comment = 'From: Jack\nI bought some boots.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.239757
1    0.278574
2    0.261081
3    0.232894
4    0.274690
5    0.538379
6    0.334921
Name: embedding, dtype: float64
Similar task(s):
{
    "5":{
        "completed":false,
        "topic":"Shopping",
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    },
    "6":{
        "completed":false,
        "topic":"Grocery Shopping",
        "type":"action_item",
        "due date":"2024-05-14",
        "requestor":"Jack",
        "actor":"Silvia",
        "summary":"Buy eggs",
        "notes":"Silvia needs to buy eggs"
    }
}
Updated task(s):
{
    "5":{
        "completed":true,
        "topic":"Shopping",
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks. *Bought b

In [21]:
comment = 'From: Jack\nI finished the task due on 04/14'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.306569
1    0.384165
2    0.374807
3    0.303251
4    0.474481
5    0.430495
6    0.390500
Name: embedding, dtype: float64
Similar task(s):
{
    "0":{
        "completed":false,
        "topic":"Probate Process - Truist Bank Claim",
        "type":"question",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summary":"Confirmation of payment to Truist Bank and forwarding of Satisfaction and Release of Claim",
        "notes":"Have you made the payment to Truist? If so, have they sent you a Satisfaction and Release of Claim? Please forward to us that was sent to you so we can file it. If not, and payment was made, let us know if you would like us to contact them on this."
    },
    "1":{
        "completed":false,
        "topic":"Probate Process - Safe Deposit Box Inventory",
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summa

In [22]:
comment = 'From: Jack\nActually, Nick should buy the eggs.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.209107
1    0.301732
2    0.295489
3    0.250507
4    0.285226
5    0.403069
6    0.508454
Name: embedding, dtype: float64
Similar task(s):
{
    "1":{
        "completed":false,
        "topic":"Probate Process - Safe Deposit Box Inventory",
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summary":"Confirm contents removal and closure of accounts\/boxes",
        "notes":"Please confirm that the contents have been removed and the accounts\/boxes have been closed."
    },
    "5":{
        "completed":false,
        "topic":"Shopping",
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    },
    "6":{
        "completed":false,
        "topic":"Grocery Shopping",
        "type":"action_item",
        "due date":"2024-05-14",
        

In [23]:
comment = 'From: Jack\nI just love zebras!'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.137839
1    0.132842
2    0.141251
3    0.157941
4    0.174524
5    0.271059
6    0.159191
Name: embedding, dtype: float64
Similar task(s):
{

}
Updated task(s):
{

}


In [24]:
comment = 'From: Jack\nI need to hang out with Kodak soon.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.199121
1    0.273477
2    0.248887
3    0.212086
4    0.285128
5    0.380622
6    0.299351
Name: embedding, dtype: float64
Similar task(s):
{
    "5":{
        "completed":false,
        "topic":"Shopping",
        "type":"action_item",
        "due date":"2024-04-14",
        "requestor":"Jack",
        "actor":"Jack",
        "summary":"Buy shoes",
        "notes":"I need to buy some cool new kicks."
    }
}
Updated task(s):
{

}


In [26]:
comment = 'From: Jack\nMerge tasks 2 and 3 about the probate process estate accounts.'

print('similarity scores:')
print(todos.get_task_similarity(comment))

similar_tasks_json = todos.get_similar_tasks_as_json(comment, 0.3)
print('Similar task(s):')
print(similar_tasks_json)

updated_tasks_df = sb.update_tasks(comment, similar_tasks_json)
print('Updated task(s):')
print(updated_tasks_df.to_json(orient='index', indent=4))

similarity scores:
0    0.483500
1    0.555472
2    0.605566
3    0.560612
4    0.556518
5    0.318824
6    0.299083
Name: embedding, dtype: float64
Similar task(s):
{
    "0":{
        "completed":false,
        "topic":"Probate Process - Truist Bank Claim",
        "type":"question",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summary":"Confirmation of payment to Truist Bank and forwarding of Satisfaction and Release of Claim",
        "notes":"Have you made the payment to Truist? If so, have they sent you a Satisfaction and Release of Claim? Please forward to us that was sent to you so we can file it. If not, and payment was made, let us know if you would like us to contact them on this."
    },
    "1":{
        "completed":false,
        "topic":"Probate Process - Safe Deposit Box Inventory",
        "type":"action_item",
        "due date":null,
        "requestor":"Marcella Brodhurst",
        "actor":"Jack",
        "summa